In [1]:
cd ..

C:\Users\rajag\Desktop\convolve\convolve_epoch_1


In [2]:
cd data

C:\Users\rajag\Desktop\convolve\convolve_epoch_1\data


In [3]:
# import libraries
import pandas as pd 
import numpy as np
import json

In [4]:
# custom settings 
pd.set_option('display.max_colwidth', None)

In [5]:
# get the data 
with open('train.json', 'r') as f:
    dict_train = json.load(f)

df = pd.DataFrame.from_dict(dict_train, orient='index')
df.reset_index(level=0, inplace=True)
df.rename(columns = {'index':'log',0:'status'}, inplace = True)

In [23]:
# Prepare the training data
df_parsed = df.copy(deep=True)
df_parsed["log"]= df_parsed["log"].str.split(":", n = 3, expand = False)
df_temp = df_parsed.copy(deep=True)
df_temp = pd.DataFrame(df_parsed['log'].to_list(), columns=['log1','log2','log3','log4'])
df_temp['status'] = df_parsed['status']
df4 = df_temp[df_temp['log4'].isnull()].copy(deep=True)
df5 = df_temp[df_temp['log4'].notnull()].copy(deep=True)
train1 = df4.drop(['log4', 'log1', 'log2'], axis=1)
train2 = df5.drop(['log3', 'log1', 'log2'], axis=1)
train1.rename(columns = {'log3':'log'}, inplace = True)
train2.rename(columns = {'log4':'log'}, inplace = True)
tmp_list = [train1,train2]
train = pd.concat(tmp_list)
train = train.dropna()

# Method 1

In [24]:
# https://www.analyticsvidhya.com/blog/2022/02/sentiment-analysis-using-transformers/

In [25]:
train1 = train.copy(deep=True)

In [26]:
train1["status"] = train1["status"].map({"abnormal": 0, "normal": 1})

In [27]:
# method 5, custom sampler
total_normal = train1[train1["status"] == 1]
total_abnormal = train1[train1["status"] == 0]

sampled_total_normal = total_normal.sample(n=69692)

normal_distributed_df = pd.concat([total_abnormal, sampled_total_normal])

balanced_train2 = normal_distributed_df.sample(frac=1,random_state=42)

In [28]:
balanced_train2.head()

,log,status
3360387,"J10-U01 RAS KERNEL INFO CE sym 28, at 0x144c5720, mask 0x01\n",1
3683334,J02-U11 RAS KERNEL INFO\n,1
4150103,J16-U01 RAS KERNEL FATAL data TLB error interrupt,0
4087324,J16-U01 RAS KERNEL FATAL data TLB error interrupt,0
4140189,J16-U01 RAS KERNEL FATAL data TLB error interrupt,0


In [29]:
pip install transformers